# Demo

###### Imports

In [1]:
# This Python file uses the following encoding: utf-8
from lib.predict_pdf import predict_documents_pdf, predictions_dataframe, load_prediction_models
from lib.utils import move_flattened_files
from lib.untar import ExtractNested
import os

Using TensorFlow backend.


###### Input arguments

In [2]:
pdf_folder = '../files_to_predict'
csv_out_file = '../predictions2.csv'

do_ocr = False
extract_from_tar = False
tar_root = '../prova.tar.gz'
use_a05_model = True

###### Tar extraction (execute only the first time if starting from a tar)

In [ ]:
# Accetto solo file con dimensione < 1MB e che finiscono per 001.pdf (si può cambiare a piacere il filtro qui)
def file_filter(f):
    return str(f).endswith(u'001.pdf') and os.path.getsize(str(f))<(1024**2)

In [ ]:
# Solo se si deve partire da una tar e non si è già estratto
if extract_from_tar and not os.path.exists(tar_root[:-7]):
    ExtractNested(tar_root)
    move_flattened_files(tar_root[:-7], pdf_folder, file_filter)

###### Load models

In [3]:
models_original = {'gensim_file':'models/gensim_model_5000.d2v', 
                  'keras_model_file':'models/keras_model.json',
                  'keras_weights_file':'models/keras_weights_5000.h5',
                  'permitted_words_file':'first_5000_words.json'}

model_with_a05 = {'gensim_file':'models/gensim_5000_model_with_verb.d2v', 
                  'keras_model_file':'models/keras_model_retry3.json',
                  'keras_weights_file':'models/keras_weights_verb_retry3.h5',
                  'permitted_words_file':'first_5000_words_with_verb_cost.json'}
'''
model_with_a05_experimental = {'gensim_file':'models/gensim_5000_model_with_verb.d2v', 
                  'keras_model_file':'models/keras_model_deep.json',
                  'keras_weights_file':'models/keras_weights_verb_deep.h5',
                  'permitted_words_file':'first_5000_words_with_verb_cost.json'}
'''

# Change the model here
models_demo = model_with_a05 if use_a05_model else models_original

predict_pdfs = load_prediction_models(**models_demo)

### Prediction

In [4]:
pdf_names = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith("pdf")]
#pdf_names = ['../files_to_predict/4908476390001.pdf']
predictions, filtered_filenames = predict_pdfs(pdf_names, do_ocr=do_ocr)

#### Save predictions to csv

In [5]:
def basenames(files):
    return [os.path.basename(f) for f in files]

In [6]:
df = predictions_dataframe(basenames(pdf_names), basenames(filtered_filenames), predictions, csv_out_file)
#df.to_csv(csv_out_file,  sep=',', index=False, encoding='utf-8')

In [7]:
df

,Nome file,Errore,Messaggio errore,Output rete,Predizione
0,5162093290002.pdf,No,,0.685052,COSTITUTIVO
1,5162128590002.pdf,No,,0.0284682,NON COSTITUTIVO
2,5162128480002.pdf,No,,0.658475,COSTITUTIVO
3,5162167840002.pdf,No,,0.651958,COSTITUTIVO
4,5162144110002.pdf,No,,0.665002,COSTITUTIVO
5,5162128480001.pdf,No,,0.173126,NON COSTITUTIVO
6,5162140190002.pdf,Si,scansione,,
7,5162093290001.pdf,Si,scansione,,
8,5162140190001.pdf,Si,scansione,,
9,5162167840001.pdf,Si,scansione,,


In [ ]:
len(df.loc[df["Predizione"]=="NON COSTITUTIVO"])

In [ ]:
df.loc[df["Nome file"] == "5162093290002.pdf"]

In [ ]:
df.loc[df["Nome file"] == "5162128480001.pdf"]

In [ ]:
df.loc[df["Nome file"] == "5162128480002.pdf"]